In [9]:
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import time
import os

In [10]:

# ## Fetch Crypto Data

def fetch_crypto_data(symbol, interval, limit=100, retries=3, delay=2):
    url = f"https://api.binance.us/api/v3/klines?symbol={symbol}USDT&interval={interval}&limit={limit}"
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                             'quote_asset_volume', 'trades', 'taker_buy_base', 'taker_buy_quote', 'ignore'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df['close'] = df['close'].astype(float)
            return df[['close']]
        except requests.exceptions.RequestException as e:
            print(f"API Error (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(delay)
    print("Failed to fetch data after retries.")
    return None

In [11]:
# ## Prepare Data

# %%
def prepare_data(df, time_steps=20):
    if len(df) <= time_steps:
        raise ValueError(f"Dataset too small for time_steps={time_steps}. Needs at least {time_steps + 1} rows.")
    
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    
    X, y = [], []
    for i in range(len(df_scaled) - time_steps):
        X.append(df_scaled[i:i + time_steps])
        y.append(df_scaled[i + time_steps])
    
    X, y = np.array(X), np.array(y)
    return X, y, scaler

In [12]:
# ## Build GRU Model

# %%
def build_gru_model(input_shape):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        GRU(50, return_sequences=False, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [13]:
def train_and_save_model(symbol, interval, epochs=100, batch_size=16, time_steps=20, model_filename="gru_model.h5"):
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        return None
    
    X, y, scaler = prepare_data(df, time_steps)
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    model = build_gru_model((X.shape[1], X.shape[2]))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Add model checkpoint callback to save the best model
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        model_filename, monitor='val_loss', save_best_only=True, mode='min', verbose=1
    )
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,
              callbacks=[early_stopping, model_checkpoint])
    
    # Load the best model after training finishes
    model = load_model(model_filename)
    
    model.save(model_filename)
    scaler_filename = model_filename.replace(".h5", "_scaler.pkl")
    joblib.dump(scaler, scaler_filename)
    print(f"Model saved as {model_filename}, Scaler saved as {scaler_filename}")
    return model_filename, scaler_filename


In [14]:
# ## Evaluate Model

# %%
def evaluate_model(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_original = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test_original, predictions))
    mae = mean_absolute_error(y_test_original, predictions)
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")


In [15]:
# ## Load Model and Predict

# %%
def load_and_predict(model_filename, symbol, interval, time_steps=20):
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        return None
    
    X, _, scaler = prepare_data(df, time_steps)
    model = load_model(model_filename)
    predictions = model.predict(X)
    predictions = scaler.inverse_transform(predictions)
    
    return df.index[-len(predictions):], predictions

In [16]:
# ======================= Run Training ======================= #
if __name__ == "__main__":
    # Set Parameters
    symbol = "BTC"
    interval = "1h"
    epochs = 10
    batch_size = 8
    time_steps = 20
    model_filename = "gru_model.h5"

    # Train Model
    model_path, scaler_path = train_and_save_model(symbol, interval, epochs, batch_size, time_steps, model_filename)

    # Evaluate Model
    if model_path:
        print("🎯 Model training completed successfully!")
    else:
        print("❌ Model training failed.")

Epoch 1/10


c:\Users\yasir\Desktop\Python Projects\LSTM\lenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.8910
Epoch 1: val_loss improved from inf to 0.74353, saving model to gru_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - loss: 0.8591 - val_loss: 0.7435
Epoch 2/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7132
Epoch 2: val_loss improved from 0.74353 to 0.61819, saving model to gru_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.7017 - val_loss: 0.6182
Epoch 3/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6145
Epoch 3: val_loss improved from 0.61819 to 0.54887, saving model to gru_model.h5


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.3673 - val_loss: 0.3300
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3375
Epoch 8: val_loss improved from 0.33001 to 0.28766, saving model to gru_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3359 - val_loss: 0.2877
Epoch 9/10
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3051
Epoch 9: val_loss improved from 0.28766 to 0.25741, saving model to gru_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.3003 - val_loss: 0.2574
Epoch 10/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2795
Epoch 10: val_loss improved from 0.25741 to 0.22534, saving model to gru_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2669 - val_loss: 0.2253


Model saved as gru_model.h5, Scaler saved as gru_model_scaler.pkl
🎯 Model training completed successfully!
